In [1]:
script_version='2.6'
date='March 8th, 2018, UNB'
import gzip
import queue
import numpy as np

import pandas
import numpy

method1_1="LifeModel_Binary"
method1_2="Fixed_Binary"
method1_3="Reg_Binary"

method2_1="LifeModel_Multiple"
method2_2="Fixed_Multiple"
method2_3="Reg_Multiple"

method3_1="LifeModel_Accel"
method3_2="Fixed_Accel"
method3_3="Reg_Accel"
method3_4="LifeModel_Accel_AutoShift_reg"

method4_1="reg_LifeModel32AutoShift"
method4_2="reg_Fixed32AutoShift"
#method4_3="reg_LifeModel32"
#method4_4="reg_Fixed32"

input_file_variable_name = ""
#Possible values:
#1               "_50Miss"
#2               "_10Miss"
#3               ""

method = method4_2

omit=False
#method=method1_2

reg=False
    
if(method==method4_1 or
   method==method4_2 or
   method==method4_3 or
   method==method4_4):
    reg=True

fall_detection_mode= "two_step"
# single_step
# two_step
    
X=[]
Y=[]
if(not omit):
    limit=34100
    max=34100
else:
    limit=5200
    max=5200

F=8371
K=32

accelData=False
if(method==method3_1 or method==method3_2 or method==method3_3 or method==method3_4 or method==method4_1 or method==method4_2):
    limit=max=70
    F=4
    K=32
    accelData=True

current_total=0

#data=[]

#diagnosis_classes=10
#curDir='C:\\Users\\manas\\OneDrive\\@Dev\\SP4\\PythonCNTKVS2017SP4\\PythonCNTKVS2017SP4\\'
curDir=""

if(reg):
    if(method==method4_1):
        filename=curDir+'FallDataAvgLifeModel32AutoShift{0}_2018-03-10  15-17-39.csv'
    elif(method==method4_2):
        filename=curDir+'FallDataAvgFixed32AutoShift{0}_2018-03-10  15-17-39.csv'
        
    print("Opened input file and ready for reading from {0}".format(filename.format(input_file_variable_name)))
    
else:
    if(method==method1_1):#LifeModel
        filename=curDir+'MIMICIII_Diag_Proc_LM_{0}12-9-2017 7-10-28 PM.csv'
    elif (method==method1_2):##Fixed
        filename=curDir+'MIMICIII_Diag_Proc_Fixed_{0}12-10-2017 8-41-34 PM.csv'
    elif (method==method1_3):##Reg
        filename=curDir+'All Patients Reg Fixed\MIMICIII_Diag_Proc_Reg_{0}1-9-2018 3-51-30 PM.csv'

    ##Omit Life Model multiple output
    if(method==method2_1):#LifeModel
        filename=curDir+"MIMICIII_Diag_Proc_LM_Omit_{0}1-10-2018 2-20-50 PM.csv"
    elif (method==method2_2):##Fixed
        filename=curDir+""
    elif (method==method2_3):##Reg
        filename=curDir+'\All Patients Reg Fixed\MIMICIII_Diag_Proc_Reg_{0}1-9-2018 3-51-30 PM.csv'

    if(method==method3_1):#LifeModel
        filename=curDir+"FallDataAvgLifeModel32{0}.csv"
    elif (method==method3_2):##Fixed
        filename=curDir+"FallDataAvgFixed32{0}.csv"
    elif (method==method3_4):#mehrdad
        filename=curDir+"FallDataAvgLifeModel32AutoShift{0}.csv"

        
    print("Opened input file and ready for reading from {0}".format(filename.format("Input")))
    #print("Read input samples: {0} samples from {1}".format(len(X),filename.format("Input")))
    
if(accelData):
    file_in=open(curDir+filename.format(input_file_variable_name))
else:#Normal Mortality
    file_in=gzip.open(curDir+filename.format("Input")+'.gz')

Opened input file and ready for reading from FallDataAvgFixed32AutoShift_2018-03-10  15-17-39.csv


In [2]:
#FallDataAvgFixed32_2018-03-10  15-17-39 -->                 (70, 129)
#FallDataAvgFixed32AutoShift_10Miss_2018-03-10  15-17-39 --> (458, 130)
#FallDataAvgFixed32AutoShift_50Miss_2018-03-10  15-17-39 --> (453, 130)
#FallDataAvgFixed32AutoShift_2018-03-10  15-17-39 -->        (460, 130)

#FallDataAvgLifeModel32_2018-03-10  15-17-39 -->                 (70, 129)
#FallDataAvgLifeModel32AutoShift_10Miss_2018-03-10  15-17-39 --> (458, 130)
#FallDataAvgLifeModel32AutoShift_50Miss_2018-03-10  15-17-39 --> (453, 130)
#FallDataAvgLifeModel32AutoShift_2018-03-10  15-17-39 -->        (460, 130)

In [3]:
def file_read_output(fname, future=None, reg_read=False):
    if(not reg_read):
        with open(fname) as f: 
                counter=0
                if future:
                    for line in f:
                        if(not omit):
                            #Y.append(int(line.split(',')))#Check if this is not correct
                            Y.append(int(line))
                        else:
                            lines=line.split(',') 
                            lines=list(map(lambda x: int(x),lines))
                            #Y.append(lines)
                            Y.append(sum(lines)/5)#Make it a 5 category thing, regression
                        counter+=1
                        if(counter==limit):
                            break
                        #lines=line.split(',')                                                
                        #lines=list(map(lambda x: float(x),lines))
                        #Y.append([lines[i*F:(i+1)*F] for i in range(K)])
                        
                    f.close()#Y.append(data[:])
                else: #History                    
                    for line in f:  
                        lines=line.split(',')                                                
                        lines=list(map(lambda x: float(x),lines))
                        X.append([lines[i*F:(i+1)*F] for i in range(K)])
                        counter+=1
                        print("Reading patient "+str(counter))
                        if(counter==limit):
                            break
                        #print("X Shape "+str(len(X[0][0])))
                    #X.append(data[:])
                    #print(len(X))
                        #Y.append([int(j == int(lines[0])) for j in range(diagnosis_classes)])
                        #Y.append(lines[0])
                        #X.append([lines[1+i*25:1+(i+1)*25] for i in range(25)])
    else:
        print("MehrdadRegTest22")
        df = np.array(pandas.read_csv(fname, usecols=[1], header=None))
        print(df.shape)
        for _ in df[:,0].tolist():
            if(fall_detection_mode=="two_step"):
                if(_==0):
                    Y.append(_)
            else:    
                Y.append(_)# :( 

In [4]:
#autoShifted regression
if (reg):
    file_read_output(filename.format(input_file_variable_name),True, True) 
    print("Read output samples: {0} samples from {1}".format(len(Y),filename.format(input_file_variable_name)))
else:
    file_read_output(filename.format("Output"),True, False) #Replace Type with output for reading future
    #file_out=open(filename.format("Output"))
    print("Read output samples: {0} samples from {1}".format(len(Y),filename.format("Output")))

MehrdadRegTest22
(460, 1)
Read output samples: 70 samples from FallDataAvgFixed32AutoShift_2018-03-10  15-17-39.csv


In [5]:
len(Y)

70

In [6]:
'''
def batch_read(size=1):    
    global current_total, X, file_in
    X=[]
    counter=0    
    for c in range(size):
        if(current_total<max):
            line=next(file_in)
            if(accelData):
                linesB=line                
            else:#Normal
                linesB=line.decode('UTF-8')            
            lines=linesB.split(',')                
            lines=list(map(lambda x: float(x),lines[:-1]))
            #print("Len of lines: {0}".format(len(lines)))
            X.append([lines[i*F:(i +1)*F] for i in range(K)])
            #print("Len of X: {0}".format(len(X)))
            counter+=1
            print("Reading patient "+str(counter),end="\r",flush=True)
            if(counter==limit):
                break
            current_total+=1
        else:
            print('Can not read more than {0} samples.'.format(current_total))
            return X
            break
    print("**Finished Reading {0} Samples**".format(size))
    return X
'''

'\ndef batch_read(size=1):    \n    global current_total, X, file_in\n    X=[]\n    counter=0    \n    for c in range(size):\n        if(current_total<max):\n            line=next(file_in)\n            if(accelData):\n                linesB=line                \n            else:#Normal\n                linesB=line.decode(\'UTF-8\')            \n            lines=linesB.split(\',\')                \n            lines=list(map(lambda x: float(x),lines[:-1]))\n            #print("Len of lines: {0}".format(len(lines)))\n            X.append([lines[i*F:(i +1)*F] for i in range(K)])\n            #print("Len of X: {0}".format(len(X)))\n            counter+=1\n            print("Reading patient "+str(counter),end="\r",flush=True)\n            if(counter==limit):\n                break\n            current_total+=1\n        else:\n            print(\'Can not read more than {0} samples.\'.format(current_total))\n            return X\n            break\n    print("**Finished Reading {0} Samples*

In [7]:
def batch_read_thread(q, size=2):    
    global current_total, X, file_in
    X=[]
    counter=0    
    for c in range(size):
        if(current_total<max):
            line=next(file_in)
            if(accelData):
                linesB=line                
            else:#Normal
                linesB=line.decode('UTF-8')     
            lines=linesB.split(',')             
            lines=list(map(lambda x: float(x),lines))
            #print("Len of lines: {0}".format(len(lines)))
            '''
            if(reg):
                if(fall_detection_mode=='two_step'):
                    if(lines[0]==0):
                        X.append([lines[i*F+2:(i +1)*F+2] for i in range(K)])
                else:
                    X.append([lines[i*F+2:(i +1)*F+2] for i in range(K)])
            else:'''
            X.append([lines[i*F+2:(i +1)*F+2] for i in range(K)])
            print(lines[0])
            print("$$$$$$$$$$$$$$$$$$$$$$$$$$$")
            #print("Len of X: {0}".format(len(X)))            
            #print("F is :"+str(F))
            #print("Array Shape:")

            #print(np.array(X).shape)
            print("  *******I/O("+str(counter+1)+'/'+str(size)+'->'+str(current_total+1),end=")***\r",flush=True)
            counter+=1
            if(counter==limit):
                break
            current_total+=1
        else:
            print('Can not read more than {0} samples.'.format(current_total))
            X2=X
            q.put(X2)
            return X2
            break
    print("**Finished Reading {0} from {1} to {2} Samples***".format(size,current_total-size,current_total))
    X2=X
    q.put(X2)
    return X2

In [8]:
def reset():
    global current_total, X, file_in
    file_in.close()
    if(accelData):
        file_in=open(curDir+filename.format(input_file_variable_name))
    else:
        file_in=gzip.open(curDir+filename.format("Input")+'.gz')
    current_total=0
    X=[]
    counter=0

In [9]:
#import pandas as pd

In [10]:
#dftest = pd.read_csv("FallDataAvgFixed32_2018-03-10  15-17-39.csv", header=None)

In [11]:
#FallDataAvgFixed32_2018-03-10  15-17-39 -->                 (70, 129)
#FallDataAvgFixed32AutoShift_10Miss_2018-03-10  15-17-39 --> (458, 130)
#FallDataAvgFixed32AutoShift_50Miss_2018-03-10  15-17-39 --> (453, 130)
#FallDataAvgFixed32AutoShift_2018-03-10  15-17-39 -->        (460, 130)

#FallDataAvgLifeModel32_2018-03-10  15-17-39 -->                 (70, 129)
#FallDataAvgLifeModel32AutoShift_10Miss_2018-03-10  15-17-39 --> (458, 130)
#FallDataAvgLifeModel32AutoShift_50Miss_2018-03-10  15-17-39 --> (453, 130)
#FallDataAvgLifeModel32AutoShift_2018-03-10  15-17-39 -->        (460, 130)

In [12]:
X

[]

In [13]:
len(Y)

70